In [ ]:
pip install requests

In [ ]:
#apy key needed

import requests
import json

base_url = "https://www.carboninterface.com/api/v1"
api_key = "//APY KEY//"


headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}
data = {
    "type": "flight",
    "passengers": 2,
    "legs": [
        {"departure_airport": "MAD", "destination_airport": "CDG"},
    ]
}

response = requests.post(f"{base_url}/estimates", headers=headers, data=json.dumps(data))

if response.status_code == 201:
    result = response.json()
    print("Estimación de emisiones de carbono para vuelo España-Francia:")
    print(f"ID de la estimación: {result['data']['id']}")
    print(f"Tipo: {result['data']['type']}")
    print(f"Atributos:")
    print(f"  Pasajeros: {result['data']['attributes']['passengers']}")
    # print(f"  Emisiones en libras: {result['data']['attributes']['carbon_lb']}")
    print(f"  Emisiones en kilogramos: {result['data']['attributes']['carbon_kg']}")
    print(f"  Emisiones en toneladas métricas: {result['data']['attributes']['carbon_mt']}")
    print(f"  Distancia en millas: {result['data']['attributes']['distance_value']}")
else:
    print(f"Error en la solicitud: {response.status_code}")
    print(response.text)

In [ ]:
#find all the brands

response = requests.get(f"{base_url}/vehicle_makes", headers=headers)

if response.status_code == 200:
    vehicle_makes = response.json()
    for make in vehicle_makes:
        print(f"Marca: {make['data']['attributes']['name']}, ID: {make['data']['id']}")
else:
    print(f"Error al obtener las marcas de vehículos: {response.status_code}")


In [ ]:
#find all the models in a brand

make_id = "056c60f6-2aaf-4a90-b3e4-d401c79da2ae"
response = requests.get(f"{base_url}/vehicle_makes/{make_id}/vehicle_models", headers=headers)

if response.status_code == 200:
    vehicle_models = response.json()
    for model in vehicle_models:
        print(f"Modelo: {model['data']['attributes']['name']}, ID: {model['data']['id']}")
else:
    print(f"Error al obtener los modelos: {response.status_code}")


In [ ]:
#number of brands, and number of models in each brand

import requests

base_url = "https://www.carboninterface.com/api/v1"
api_key = "vJxBNzJxpgaG4mS9wrqTNw"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Fetch vehicle makes
response = requests.get(f"{base_url}/vehicle_makes", headers=headers)

if response.status_code == 200:
    vehicle_makes = response.json()
    num_brands = len(vehicle_makes)
    print(f"Number of brands: {num_brands}")
    
    brand_model_counts = {}
    
    # Fetch models for each brand
    for make in vehicle_makes:
        make_id = make['data']['id']
        make_name = make['data']['attributes']['name']
        
        response = requests.get(f"{base_url}/vehicle_makes/{make_id}/vehicle_models", headers=headers)
        if response.status_code == 200:
            vehicle_models = response.json()
            num_models = len(vehicle_models)
            brand_model_counts[make_name] = num_models
            print(f"Brand: {make_name}, Number of models: {num_models}")
        else:
            print(f"Error fetching models for brand {make_name}: {response.status_code}")
else:
    print(f"Error fetching vehicle makes: {response.status_code}")


In [ ]:
#to know the attributes of the car

import requests

base_url = "https://www.carboninterface.com/api/v1"
api_key = "vJxBNzJxpgaG4mS9wrqTNw"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Function to get vehicle makes
def get_vehicle_makes():
    response = requests.get(f"{base_url}/vehicle_makes", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching vehicle makes: {response.status_code}")
        return []

# Function to get vehicle models for a given make
def get_vehicle_models(make_id):
    response = requests.get(f"{base_url}/vehicle_makes/{make_id}/vehicle_models", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching vehicle models for make {make_id}: {response.status_code}")
        return []

# Print attributes of each vehicle model
def print_vehicle_attributes(make_id):
    vehicle_models = get_vehicle_models(make_id)
    for model in vehicle_models:
        model_id = model['data']['id']
        model_attributes = model['data']['attributes']
        print(f"Model ID: {model_id}")
        for key, value in model_attributes.items():
            print(f"{key}: {value}")
        print("\n")

vehicle_makes = get_vehicle_makes()

for make in vehicle_makes:
    make_id = make['data']['id']
    make_name = make['data']['attributes']['name']
    print(f"\nVehicle Make: {make_name}")
    print_vehicle_attributes(make_id)


In [ ]:
# to calculate the train emissions, retry

def calculate_train_emissions(distance, train_type='electric', passengers=1):
    emission_factors = {'electric': 0.04, 'diesel': 0.14}
    factor = emission_factors.get(train_type.lower(), 0.04)
    return distance * factor * passengers

# Example usage for a train trip
train_distance = 300  # Example distance in miles
train_emissions = calculate_train_emissions(train_distance, train_type='diesel', passengers=3)
print(f"Train emissions for {train_distance} miles: {train_emissions} kg CO₂")


In [ ]:
#only for modern cars

import requests
import json
import pandas as pd

base_url = "https://www.carboninterface.com/api/v1"
api_key = "aPnLmPUrOvAjoA5FxkOwtA"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Function to get vehicle makes
def get_vehicle_makes():
    response = requests.get(f"{base_url}/vehicle_makes", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching vehicle makes: {response.status_code}")
        return []

# Function to get vehicle models for a given make
def get_vehicle_models(make_id):
    response = requests.get(f"{base_url}/vehicle_makes/{make_id}/vehicle_models", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching vehicle models for make {make_id}: {response.status_code}")
        return []

# Function to get emissions estimate for a vehicle model
def get_emissions(vehicle_model_id):
    data = {
        "type": "vehicle",
        "distance_unit": "km",
        "distance_value": 100,  # Estimating for a 100km trip
        "vehicle_model_id": vehicle_model_id
    }
    response = requests.post(f"{base_url}/estimates", headers=headers, data=json.dumps(data))
    if response.status_code == 201:
        return response.json()
    else:
        print(f"Error fetching emissions for model {vehicle_model_id}: {response.status_code}")
        return None

# Get vehicle makes and limit to top 10
vehicle_makes = get_vehicle_makes()[:10]
data_list = []

for make in vehicle_makes:
    make_id = make['data']['id']
    make_name = make['data']['attributes']['name']
    vehicle_models = get_vehicle_models(make_id)
    
    # Filter models from the last 10 years
    recent_models = [model for model in vehicle_models if model['data']['attributes']['year'] >= 2021][:2]
    
    for model in recent_models:
        model_id = model['data']['id']
        emissions = get_emissions(model_id)
        if emissions:
            attributes = emissions['data']['attributes']
            data_list.append({
                "distance_value": attributes.get("distance_value"),
                "vehicle_make": make_name,
                "vehicle_model": attributes.get("vehicle_model"),
                "vehicle_year": attributes.get("vehicle_year"),
                "vehicle_model_id": model_id,
                "distance_unit": attributes.get("distance_unit"),
                "estimated_at": attributes.get("estimated_at"),
                "carbon_g": attributes.get("carbon_g"),
                "carbon_lb": attributes.get("carbon_lb"),
                "carbon_kg": attributes.get("carbon_kg"),
                "carbon_mt": attributes.get("carbon_mt")
            })

# Create a DataFrame
df = pd.DataFrame(data_list)
print(df)

# Optionally, save to a CSV file
df.to_csv("vehicle_emissions_data_2024.csv", index=False)


In [ ]:
import requests
import json
import pandas as pd

base_url = "https://www.carboninterface.com/api/v1"
api_key = "0jft2toWfGP0pbp5BGOrA"
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# Function to get vehicle makes
def get_vehicle_makes():
    response = requests.get(f"{base_url}/vehicle_makes", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching vehicle makes: {response.status_code}")
        return []

# Function to get vehicle models for a given make
def get_vehicle_models(make_id):
    response = requests.get(f"{base_url}/vehicle_makes/{make_id}/vehicle_models", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching vehicle models for make {make_id}: {response.status_code}")
        return []

# Function to get emissions estimate for a vehicle model
def get_emissions(vehicle_model_id):
    data = {
        "type": "vehicle",
        "distance_unit": "km",
        "distance_value": 100,  # Estimating for a 100km trip
        "vehicle_model_id": vehicle_model_id
    }
    response = requests.post(f"{base_url}/estimates", headers=headers, data=json.dumps(data))
    if response.status_code == 201:
        return response.json()
    else:
        print(f"Error fetching emissions for model {vehicle_model_id}: {response.status_code}")
        return None

# Get vehicle makes and filter by specified brands
brands = ["Tesla", "Lamborghini", "Ford", "Toyota", "Hyundai"]
vehicle_makes = get_vehicle_makes()
selected_makes = [make for make in vehicle_makes if make['data']['attributes']['name'] in brands]

data_list = []

for make in selected_makes:
    make_id = make['data']['id']
    make_name = make['data']['attributes']['name']
    vehicle_models = get_vehicle_models(make_id)
    
    # Filter models for new cars (as close to 2024 as possible) and old cars (as close to 1990 as possible)
    if make_name == "Hyundai":
        new_models = [model for model in vehicle_models if model['data']['attributes']['year'] >= 2020 and "hybrid" in model['data']['attributes']['name'].lower()][:5]
    else:
        new_models = [model for model in vehicle_models if model['data']['attributes']['year'] >= 2020][:5]
    old_models = [model for model in vehicle_models if model['data']['attributes']['year'] <= 1990][:5]
    
    for model in new_models + old_models:
        model_id = model['data']['id']
        emissions = get_emissions(model_id)
        if emissions:
            attributes = emissions['data']['attributes']
            data_list.append({
                "distance_value": attributes.get("distance_value"),
                "vehicle_make": make_name,
                "vehicle_model": attributes.get("vehicle_model"),
                "vehicle_year": attributes.get("vehicle_year"),
                "vehicle_model_id": model_id,
                "distance_unit": attributes.get("distance_unit"),
                "estimated_at": attributes.get("estimated_at"),
                "carbon_g": attributes.get("carbon_g"),
                "carbon_lb": attributes.get("carbon_lb"),
                "carbon_kg": attributes.get("carbon_kg"),
                "carbon_mt": attributes.get("carbon_mt")
            })

# Create a DataFrame
df = pd.DataFrame(data_list)
print(df)

# Optionally, save to a CSV file
df.to_csv("vehicle_emissions_data.csv", index=False)
